# HW 6

## SQL CARTO QUERY FUNCTION

## Extra Credit: *__This notebook can be ran using the Python2 or Python3 kernel__*

In [1]:
try:
    import urllib as ulib
except ImportError:
    import urllib2 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)        
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
    except Exception:
        raise
    return response.read()

def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

# Task 1 - Familiarize with SQL Clauses

### Sort Data by start_station_id, trip duration
*Only checking trips with duration <=3 hours*

In [3]:
query = '''
SELECT * FROM fb55.citibike
WHERE tripduration>180*60
ORDER BY start_station_id, tripduration;
'''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,Atlantic Ave & Fort Greene Pl,NaN,20408,-73.976323,33647,2015-02-04 19:52:00+00,2015-02-05 05:12:00+00,83,20353,40.683826,372,Franklin Ave & Myrtle Ave,40.694528,-73.958089,20703,Customer,NaN,0
1,NaN,E 56 St & Madison Ave,NaN,11298,-73.972924,13156,2015-02-03 18:16:00+00,2015-02-03 21:55:00+00,137,11237,40.761628,137,E 56 St & Madison Ave,40.761628,-73.972924,19794,Subscriber,1990.0,1
2,NaN,Warren St & Church St,NaN,7194,-74.009106,15251,2015-02-02 14:25:00+00,2015-02-02 18:39:00+00,152,7128,40.714740,504,1 Ave & E 15 St,40.732219,-73.981656,21024,Subscriber,1970.0,1
3,NaN,LaGuardia Pl & W 3 St,NaN,15750,-73.998102,17307,2015-02-04 12:06:00+00,2015-02-04 16:55:00+00,161,15707,40.729170,375,Mercer St & Bleecker St,40.726795,-73.996951,19870,Subscriber,1994.0,1
4,NaN,E 39 St & 3 Ave,NaN,41263,-73.976049,23588,2015-02-07 11:54:00+00,2015-02-07 18:27:00+00,167,41251,40.748901,527,E 33 St & 2 Ave,40.744023,-73.976056,19539,Subscriber,1978.0,2


### Only show the top/last 10 records (aka **head** and **tail**)

In [4]:
query = '''

(SELECT * FROM fb55.citibike
ORDER BY start_station_id DESC, tripduration DESC
LIMIT 10)

UNION ALL

(SELECT * FROM fb55.citibike
ORDER BY start_station_id ASC, tripduration ASC
LIMIT 10);

'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,South End Ave & Liberty St,NaN,1576,-74.015756,7225,2015-02-01 11:46:00+00,2015-02-01 13:47:00+00,3002,1506,40.711512,499,Broadway & W 60 St,40.769155,-73.981918,21202,Customer,NaN,0
1,NaN,South End Ave & Liberty St,NaN,8891,-74.015756,2139,2015-02-03 09:33:00+00,2015-02-03 10:09:00+00,3002,8831,40.711512,510,W 51 St & 6 Ave,40.760660,-73.980420,15552,Subscriber,1958.0,1
2,NaN,South End Ave & Liberty St,NaN,23568,-74.015756,2131,2015-02-05 08:42:00+00,2015-02-05 09:17:00+00,3002,23529,40.711512,359,E 47 St & Park Ave,40.755103,-73.974987,21473,Subscriber,1985.0,1
3,NaN,South End Ave & Liberty St,NaN,10232,-74.015756,1977,2015-02-03 16:04:00+00,2015-02-03 16:37:00+00,3002,10173,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,15309,Subscriber,1969.0,1
4,NaN,South End Ave & Liberty St,NaN,27243,-74.015756,1977,2015-02-05 16:11:00+00,2015-02-05 16:44:00+00,3002,27203,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20069,Subscriber,1969.0,1


### List all unique start_station_id values

In [5]:
query = '''
SELECT DISTINCT(start_station_id) FROM fb55.citibike;
'''
table = get_data(query)
table.head()

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


### Aggregate Functions
#### Count the number of trips (aka wc -l SQL)

In [6]:
query = '''
SELECT count(cartodb_id) as trips FROM fb55.citibike;
'''

table = get_data(query)
table.head()

,trips,Unnamed: 1
0,46200,NaN


### Find the average/min/max trip duration

In [7]:
query = '''
SELECT MIN(tripduration), MAX(tripduration), AVG(tripduration) FROM fb55.citibike;
'''
table = get_data(query)
table.head()

,min,max,avg
0,60,43016,675.865823


# Task 2 - Working with date/time

### Selecting trip started of Feb-02-2015 only

In [8]:
query = '''
select *
from fb55.citibike
where starttime::date = '2015-02-02';
'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


### Selecting trips started on the weekends
What are the average trip duration during the weekends?

In [58]:
query = '''
select AVG(tripduration) as AverageTime
from fb55.citibike
where extract(DOW from starttime::date) IN (0,6);
'''
table = get_data(query)
table.head()

,averagetime,Unnamed: 1
0,662.942181,NaN


### Can you do the same for Weekday?

In [56]:
query = '''
select AVG(tripduration) as AverageTime
from fb55.citibike
where extract(DOW from starttime::date) IN (1,2,3,4,5);
'''
table = get_data(query)
table.head()

,averagetime,Unnamed: 1
0,681.052292,NaN


# Task 3 - Working with Space

### Showing the list of start station locations using GROUP BY

In [11]:
query = '''
select start_station_name 
from fb55.citibike
group by start_station_name;
'''
table = get_data(query)
table.head()

,start_station_name,Unnamed: 1
0,E 30 St & Park Ave S,NaN
1,Pearl St & Anchorage Pl,NaN
2,W 56 St & 6 Ave,NaN
3,West Thames St,NaN
4,Bank St & Washington St,NaN


### Showing the number of trips per station
... but only for stations within 500m of Time Square

In [12]:
## Used good ol PostgreSQL

query = '''
select start_station_name, count(cartodb_id) as trips

from
(select * from fb55.citibike
where ST_Distance_Sphere(ST_MakePoint(-73.9857,40.7577),ST_MakePoint(start_station_longitude,start_station_latitude))<500) as subgroup

group by start_station_name
;
'''
table = get_data(query)
table.head()

,start_station_name,trips
0,W 45 St & 8 Ave,141
1,W 42 St & 8 Ave,221
2,Broadway & W 49 St,213
3,W 43 St & 6 Ave,112
4,W 41 St & 8 Ave,507


# Task 4 - Putting it all together

Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!

In [59]:
query = '''
select * from (

select start_station_name, avg(tripduration) as Average_Trip

from
(select * from fb55.citibike
where ST_Distance_Sphere(ST_MakePoint(-73.9857,40.7577),ST_MakePoint(start_station_longitude,start_station_latitude))<500) as subgroup

where extract(DOW from starttime::date) IN (0,6)
group by start_station_name) as subgroup2


order by Average_Trip DESC
limit 1;
'''
table = get_data(query)
table.head()

,start_station_name,average_trip
0,Broadway & W 49 St,1010.104167


Extra: **create lines** for trips started from stations within 500m of Times Squares and **lasted less than 2 hours**. The number of trips per each pair of stations are output as attributes of these lines. 

In [14]:
query = '''
select ST_MakeLine(ST_MakePoint(start_station_longitude,start_station_latitude), ST_MakePoint(end_station_longitude,end_station_latitude)) as Line, count(cartodb_id) 

from 
(select * from fb55.citibike
where ST_Distance_Sphere(ST_MakePoint(-73.9857,40.7577),ST_MakePoint(start_station_longitude,start_station_latitude))<500 AND tripduration < 2*60*60) as subgroup

group by start_station_longitude,start_station_latitude,end_station_longitude,end_station_latitude;
'''
lineDF = get_data(query)
lineDF.head()

,line,count
0,010200000002000000BE51C821247F52C021994848A860...,2
1,010200000002000000860DC305E87E52C0058E5FD3DE60...,1
2,010200000002000000860DC305E87E52C0058E5FD3DE60...,1
3,01020000000200000046E82D3FEC7E52C08A99B7C9AC60...,1
4,0102000000020000005A9EA44C6C7F52C08D77EC0CF860...,5


### Notes: the next step would be to convert the 'line' values into a python/geopandas ready format. Right now it is currently stored as a WKT element which is SQL's way of storing geometries. To do this we would need to install a package, but those are not currently installed on the PUI2016_Python3 kernel. 
We would then plot the corresponding lines using geopandas